In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_dir = f"datasets/correlated"
results_dir_tanh = "results/correlated"
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]


full_config_path = "correlated_N400_p6"

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)


In [6]:
from utils.generate_data import generate_correlated_data
X_train, X_test, y_train, y_test = generate_correlated_data(n=500, p=6, random_state=42)

rmse_gauss = np.zeros(4000)
rmse_rhs = np.zeros(4000)
rmse_dhs = np.zeros(4000)
rmse_dst = np.zeros(4000)

for i in range(4000):
    rmse_gauss[i] = np.sqrt(np.mean((y_test - (tanh_fit['Gaussian tanh']['posterior'].stan_variable("output_test")[i].squeeze(-1)))**2))
    rmse_rhs[i] = np.sqrt(np.mean((y_test - (tanh_fit['Regularized Horseshoe tanh']['posterior'].stan_variable("output_test")[i].squeeze(-1)))**2))
    rmse_dhs[i] = np.sqrt(np.mean((y_test - (tanh_fit['Dirichlet Horseshoe tanh']['posterior'].stan_variable("output_test")[i].squeeze(-1)))**2))
    rmse_dst[i] = np.sqrt(np.mean((y_test - (tanh_fit['Dirichlet Student T tanh']['posterior'].stan_variable("output_test")[i].squeeze(-1)))**2))

In [ ]:
np.mean(rmse_gauss)

In [ ]:
from sklearn.metrics import mean_squared_error
from properscoring import crps_ensemble
import numpy as np
import pandas as pd

# IMPORTANT: this y_test must correspond to the same test set used to make `output_test` in Stan,
# otherwise scores won’t be comparable.
from utils.generate_data import generate_correlated_data
X_train, X_test, y_train, y_test = generate_correlated_data(n=500, p=6, random_state=42)

rows = []
for model_name, model_entry in tanh_fit.items():
    post = model_entry["posterior"]

    # (S, n_test)
    y_samps = post.stan_variable("output_test").squeeze(-1)

    # Posterior-mean predictions and RMSE
    y_mean = y_samps.mean(axis=0)                                   # (n_test,)
    rmse_post_mean = float(np.sqrt(mean_squared_error(y_test, y_mean)))

    # Per-draw RMSEs and their mean
    per_draw_rmse = np.sqrt(((y_samps - y_test[None, :])**2).mean(axis=1))  # (S,)
    rmse_draw_mean = float(per_draw_rmse.mean())

    # CRPS across the ensemble (expects shape (n_test, S))
    crps = float(np.mean(crps_ensemble(y_test, y_samps.T)))

    rows.append({
        "Model": model_name,
        "RMSE_posterior_mean": rmse_post_mean,
        "RMSE_mean_over_draws": rmse_draw_mean,
        "CRPS": crps,
        "n_draws": y_samps.shape[0]
    })

results_df = pd.DataFrame(rows).sort_values("RMSE_posterior_mean")
print(results_df)


In [8]:
from utils.robust_utils import build_pytorch_model_from_stan_sample
from utils.generate_data import generate_correlated_data
import torch 

P, H=6, 16

X_train, X_test, y_train, y_test = generate_correlated_data(n=500, p=6, random_state=42)

gauss_model = build_pytorch_model_from_stan_sample(
        tanh_fit['Gaussian tanh']['posterior'], sample_idx=0, input_dim=P, hidden_dim=H,
        output_dim=1, task="regression", activation=torch.tanh
)

rhs_model = build_pytorch_model_from_stan_sample(
        tanh_fit['Regularized Horseshoe tanh']['posterior'], sample_idx=0, input_dim=P, hidden_dim=H,
        output_dim=1, task="regression", activation=torch.tanh
)

dhs_model = build_pytorch_model_from_stan_sample(
        tanh_fit['Dirichlet Horseshoe tanh']['posterior'], sample_idx=0, input_dim=P, hidden_dim=H,
        output_dim=1, task="regression", activation=torch.tanh
)

dst_model = build_pytorch_model_from_stan_sample(
        tanh_fit['Dirichlet Student T tanh']['posterior'], sample_idx=0, input_dim=P, hidden_dim=H,
        output_dim=1, task="regression", activation=torch.tanh
)



In [9]:
import numpy as np
import torch

def get_hidden_representation(model, X_np, device="cpu"):
    """
    Returns T = activation(linear1(X)) as a NumPy array of shape (n, H).
    Assumes model has .linear1 and the activation is applied in forward.
    """
    model.eval()
    model.to(device)
    X_t = torch.tensor(X_np, dtype=torch.float32, device=device)
    with torch.no_grad():
        z1 = model.linear1(X_t)                 # pre-activation
        # Try to infer activation from model.forward by checking output
        # but we don't want to call forward since it mixes linear2; so:
        # If your model's activation is torch.tanh (as you built it), do:
        T_t = torch.tanh(z1)
    return T_t.cpu().numpy()

def quantile_bin_1d(x, n_bins=10):
    """
    Return integer bin indices 0..B-1 using quantile edges.
    B may be < n_bins if many ties; we handle that downstream.
    """
    x = np.asarray(x).ravel()
    edges = np.unique(np.quantile(x, np.linspace(0, 1, n_bins+1)))
    # if edges all equal => only one unique value
    if edges.size <= 2:
        # everything in a single bin
        b = np.zeros_like(x, dtype=np.int64)
        return b, edges
    b = np.digitize(x, edges[1:-1], right=False)
    # clip to ensure indices in [0, B-1]
    b = np.clip(b, 0, edges.size - 2)
    return b, edges

def mi_from_bincounts(bx, by):
    """
    bx, by: integer bin indices (n,)
    Returns MI in nats. Robust to degenerate cases.
    """
    bx = np.asarray(bx, dtype=np.int64)
    by = np.asarray(by, dtype=np.int64)
    n = bx.size
    if n == 0:
        return 0.0

    M = int(bx.max()) + 1
    N = int(by.max()) + 1
    if M < 2 or N < 2:
        return 0.0  # one side collapsed -> no info in this projection

    joint = np.zeros((M, N), dtype=np.float64)
    np.add.at(joint, (bx, by), 1.0)
    joint /= n

    px = joint.sum(axis=1)   # (M,)
    py = joint.sum(axis=0)   # (N,)

    # Guard: if any zero-marginals, MI=0 for this projection (degenerate)
    if not np.all(px > 0) or not np.all(py > 0):
        return 0.0

    mask = joint > 0

    # Build full broadcasted matrices BEFORE masking
    # (boolean indexing does not broadcast!)
    log_joint = np.zeros_like(joint)
    with np.errstate(divide='ignore'):
        log_joint[mask] = np.log(joint[mask])

    # Full (M,N) logs of marginals
    log_px_full = np.log(px)[:, None] + np.zeros((1, N))
    log_py_full = np.log(py)[None, :] + np.zeros((M, 1))

    mi = np.sum(joint[mask] * (log_joint[mask] - log_px_full[mask] - log_py_full[mask]))
    return float(mi)

def sliced_mi_binned(X, T, n_bins=10, n_projections=64, seed=0):
    """
    Estimate I(X;T) via random 1D projections and binned MI, averaging over
    valid projections (those that produce >=2 bins on both sides).
    """
    rng = np.random.default_rng(seed)
    X = np.asarray(X, float)
    T = np.asarray(T, float)
    n, dx = X.shape
    _, dt = T.shape

    U = rng.normal(size=(n_projections, dx))
    U /= (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    V = rng.normal(size=(n_projections, dt))
    V /= (np.linalg.norm(V, axis=1, keepdims=True) + 1e-12)

    mis = []
    skipped = 0
    for r in range(n_projections):
        x_proj = X @ U[r]
        t_proj = T @ V[r]
        # pre-check: do we get at least 2 unique quantile bins on both sides?
        bx, ex = quantile_bin_1d(x_proj, n_bins)
        by, ey = quantile_bin_1d(t_proj, n_bins)
        if (ex.size <= 2) or (ey.size <= 2) or (np.unique(bx).size < 2) or (np.unique(by).size < 2):
            skipped += 1
            continue
        mis.append(mi_from_bincounts(bx, by))

    if len(mis) == 0:
        # fall back to zero if everything degenerated (shouldn’t happen with real data)
        return 0.0, 0.0, np.array([])

    mis = np.array(mis, dtype=float)
    return float(mis.mean()), float(mis.std()), mis


In [10]:
import numpy as np
from sklearn.decomposition import PCA

def make_projections(X, T, n_random=512, nx_pca=3, nt_top=6, seed=0):
    """
    Build a fixed set of 1D projection pairs (u, v):
      - PCA directions of X (nx_pca)
      - Top-variance neurons of T (nt_top)
      - Plus n_random random unit directions for each side
    Returns U: (R, dx), V: (R, dt), where R = deterministic_pairs + n_random
    """
    rng = np.random.default_rng(seed)
    X = np.asarray(X, float); T = np.asarray(T, float)
    dx = X.shape[1]; dt = T.shape[1]

    U_list, V_list = [], []

    # Deterministic projections to hit informative directions
    if nx_pca > 0:
        X_pca = PCA(n_components=min(nx_pca, dx)).fit(X)
        U_list.extend([X_pca.components_[i] for i in range(X_pca.n_components_)])
    if nt_top > 0:
        var_T = T.var(axis=0)
        top_idx = np.argsort(var_T)[::-1][:min(nt_top, dt)]
        V_list.extend([np.eye(dt)[j] for j in top_idx])

    # Pair up deterministic lists (cross-pair min length)
    R_det = min(len(U_list), len(V_list))
    U_det = np.array(U_list[:R_det]) if R_det > 0 else np.zeros((0, dx))
    V_det = np.array(V_list[:R_det]) if R_det > 0 else np.zeros((0, dt))

    # Random projections
    U_rand = rng.normal(size=(n_random, dx))
    U_rand /= (np.linalg.norm(U_rand, axis=1, keepdims=True) + 1e-12)
    V_rand = rng.normal(size=(n_random, dt))
    V_rand /= (np.linalg.norm(V_rand, axis=1, keepdims=True) + 1e-12)

    # Concatenate
    U = np.vstack([U_det, U_rand])
    V = np.vstack([V_det, V_rand])
    R = min(U.shape[0], V.shape[0])
    U = U[:R]; V = V[:R]
    return U, V

def sliced_mi_binned_with_projections(X, T, U, V, n_bins=7, trim=0.10):
    """
    Compute sliced binned MI with a FIXED set of projection pairs (U, V).
    Uses a trimmed mean across projections to reduce outlier influence.
    Returns (mean_trimmed, std_across_projections, all_projection_MIs)
    """
    X = np.asarray(X, float); T = np.asarray(T, float)
    mis = []
    for u, v in zip(U, V):
        x_proj = X @ u
        t_proj = T @ v
        bx, ex = quantile_bin_1d(x_proj, n_bins)
        by, ey = quantile_bin_1d(t_proj, n_bins)
        if (ex.size <= 2) or (ey.size <= 2) or (np.unique(bx).size < 2) or (np.unique(by).size < 2):
            continue
        mis.append(mi_from_bincounts(bx, by))
    if len(mis) == 0:
        return 0.0, 0.0, np.array([])
    mis = np.asarray(mis, float)
    # trimmed mean
    if 0.0 < trim < 0.5:
        lo = int(np.floor(trim * len(mis)))
        hi = int(np.ceil((1 - trim) * len(mis)))
        mis_sorted = np.sort(mis)
        mean_trim = mis_sorted[lo:hi].mean()
    else:
        mean_trim = mis.mean()
    return float(mean_trim), float(mis.std()), mis

def bootstrap_sliced_mi(X, T, n_bins=7, n_random=512, nx_pca=3, nt_top=6,
                        trim=0.10, B=200, seed=0):
    """
    Bootstrap MI over rows with a fixed projection set:
      1) Build fixed U,V from full (X,T)
      2) For b=1..B: resample rows with replacement, recompute sliced MI (trimmed)
    Returns dict with mean, se, ci, and raw bootstrap values.
    """
    rng = np.random.default_rng(seed)
    U, V = make_projections(X, T, n_random=n_random, nx_pca=nx_pca, nt_top=nt_top, seed=seed)

    n = X.shape[0]
    boot_vals = []
    for b in range(B):
        idx = rng.integers(0, n, size=n)
        m, _, _ = sliced_mi_binned_with_projections(X[idx], T[idx], U, V, n_bins=n_bins, trim=trim)
        boot_vals.append(m)
    boot_vals = np.asarray(boot_vals, float)

    mean_est = float(boot_vals.mean())
    se_est = float(boot_vals.std(ddof=1))
    lo, hi = np.quantile(boot_vals, [0.025, 0.975])
    return {
        "mean": mean_est,
        "se": se_est,
        "ci95": (float(lo), float(hi)),
        "boot": boot_vals,
        "U": U, "V": V
    }


In [11]:
# Get hidden representation T
T_train_gauss = get_hidden_representation(gauss_model, X_train)
T_train_rhs = get_hidden_representation(rhs_model, X_train)
T_train_dhs = get_hidden_representation(dhs_model, X_train)
T_train_dst = get_hidden_representation(dst_model, X_train)

# Standardize columns (helps binning stability)
X_std = (X_train - X_train.mean(axis=0)) / (X_train.std(axis=0) + 1e-12)
T_std_gauss = (T_train_gauss - T_train_gauss.mean(axis=0)) / (T_train_gauss.std(axis=0) + 1e-12)
T_std_rhs = (T_train_rhs - T_train_rhs.mean(axis=0)) / (T_train_rhs.std(axis=0) + 1e-12)
T_std_dhs = (T_train_dhs - T_train_dhs.mean(axis=0)) / (T_train_dhs.std(axis=0) + 1e-12)
T_std_dst = (T_train_dst - T_train_dst.mean(axis=0)) / (T_train_dst.std(axis=0) + 1e-12)

out_gauss = bootstrap_sliced_mi(
    X_std, T_std_gauss,
    n_bins=7,          # fewer bins -> lower variance
    n_random=1024,     # many projections -> stable
    nx_pca=3, nt_top=6,# deterministic informative projections
    trim=0.10,         # trimmed mean across projections
    B=50,             # bootstrap reps
    seed=123
)

out_rhs = bootstrap_sliced_mi(
    X_std, T_std_rhs,
    n_bins=7,          # fewer bins -> lower variance
    n_random=1024,     # many projections -> stable
    nx_pca=3, nt_top=6,# deterministic informative projections
    trim=0.10,         # trimmed mean across projections
    B=50,             # bootstrap reps
    seed=123
)

out_dhs = bootstrap_sliced_mi(
    X_std, T_std_dhs,
    n_bins=7,          # fewer bins -> lower variance
    n_random=1024,     # many projections -> stable
    nx_pca=3, nt_top=6,# deterministic informative projections
    trim=0.10,         # trimmed mean across projections
    B=50,             # bootstrap reps
    seed=123
)

out_dst = bootstrap_sliced_mi(
    X_std, T_std_dst,
    n_bins=7,          # fewer bins -> lower variance
    n_random=1024,     # many projections -> stable
    nx_pca=3, nt_top=6,# deterministic informative projections
    trim=0.10,         # trimmed mean across projections
    B=50,             # bootstrap reps
    seed=123
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.stats import norm

def plot_axis_aligned_ellipse(x_samples, y_samples, n_std=2.0, ax=None,
                              edgecolor='crimson', facecolor='none', alpha=0.8, lw=2, label=None):
    """
    Draw an axis-aligned ellipse centered at (mean(x), mean(y)),
    with semi-axes = n_std * (std_x, std_y).
    """
    x = np.asarray(x_samples).ravel()
    y = np.asarray(y_samples).ravel()

    x_mu, y_mu = x.mean(), y.mean()
    x_std, y_std = x.std(ddof=1), y.std(ddof=1)

    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 5))

    width  = 2 * n_std * x_std
    height = 2 * n_std * y_std

    ell = Ellipse((x_mu, y_mu), width=width, height=height, angle=0,
                  edgecolor=edgecolor, facecolor=facecolor, lw=lw, alpha=alpha, label=label)
    ax.add_patch(ell)
    ax.scatter([x_mu], [y_mu], color=edgecolor, s=40)  # mark center

    return ax

# ---- Plot all in one figure ----
fig, ax = plt.subplots(figsize=(7,6))

plot_axis_aligned_ellipse(out_gauss['boot'], rmse_gauss, n_std=2.0, ax=ax,
                          edgecolor='red', label='Gaussian')
plot_axis_aligned_ellipse(out_rhs['boot'], rmse_rhs, n_std=2.0, ax=ax,
                          edgecolor='blue', label='RHS')
plot_axis_aligned_ellipse(out_dhs['boot'], rmse_dhs, n_std=2.0, ax=ax,
                          edgecolor='green', label='DHS')
plot_axis_aligned_ellipse(out_dst['boot'], rmse_dst, n_std=2.0, ax=ax,
                          edgecolor='purple', label='DST')

ax.set_xlabel("I(X;T)")
ax.set_ylabel("RMSE")
ax.set_title("Axis-Aligned 2σ Ellipses (Uncertainty Regions)")
ax.legend()

plt.show()


## OLD

In [14]:
import numpy as np

def gen_toy_data(n=5000, sigma=0.5, seed=0, standardize=True):
    rng = np.random.default_rng(seed)
    X1 = rng.normal(size=n)
    X2 = rng.normal(size=n)
    X3 = rng.normal(size=n)  # pure noise feature, irrelevant

    # Nonlinear signal in X1,X2 only
    f = np.tanh(X1 + 0.8*X2 + 0.5*X1*X2) + 0.3*np.sin(0.7*X2 - 0.2*X1)
    y = f + rng.normal(scale=sigma, size=n)

    X = np.stack([X1, X2, X3], axis=1)

    if standardize:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-12)
        y = (y - y.mean()) / (y.std() + 1e-12)

    return X, y

X, y = gen_toy_data(n=500, sigma=0.5, seed=42)


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def generate_correlated_data_noisy(
    n, p=None, p_init=6, random_state=42, test_size=0.2,
    rho_strong=0.92, rho_weak=0.5, noise_scale=0.5,
    n_noise=0, noise_rho=0.0,
    standardize_features=True, standardize_target=True
):
    """
    Nonlinear dataset with interactions and optional extra non-influential features.

    Base structure (always present):
      - 6 features total in core set:
          X1..X4: strongly correlated block, main drivers of y (with interactions)
          X5..X6: moderately correlated weak block, small effect on y
      - y depends ONLY on X1..X6 (noise features have zero effect by construction)
      - Optional noise features (X7..X_{6+n_noise}) can be added; they are pure distractors.

    Correlation:
      - Strong block uses a latent factor with corr ~ rho_strong
      - Weak block uses a latent factor with corr ~ rho_weak
      - Noise block can be independent (noise_rho=0) or share a latent factor (noise_rho>0)

    Standardization:
      - Features are standardized columnwise if standardize_features=True
      - Target standardized if standardize_target=True

    Args:
      n: number of samples
      p: (optional) total number of features to enforce; must equal 6 + n_noise if given
      n_noise: number of additional noise features (default 0)
      noise_rho: correlation level among noise features (0 => independent)
      noise_scale: std of observation noise added to y
      test_size, random_state: as in scikit-learn

    Returns:
      X_train, X_test, y_train, y_test
    """
    rng = np.random.default_rng(random_state)
    total_p = p_init + int(n_noise)
    if p is not None and p != total_p:
        raise ValueError(f"p must equal 6 + n_noise (= {total_p}), got p={p}.")

    # Latent factors for correlation structure
    h1 = rng.normal(size=n)  # drives strong block X1..X4
    h2 = rng.normal(size=n)  # drives weak block   X5..X6

    def make_block(h, m, rho):
        eps = rng.normal(size=(n, m))
        return rho * h[:, None] + np.sqrt(max(1e-12, 1 - rho**2)) * eps

    # Core features
    X_strong = make_block(h1, 4, rho_strong)    # X1..X4
    X_weak   = make_block(h2, 2, rho_weak)      # X5..X6

    # Optional noise features (pure distractors)
    if n_noise > 0:
        if abs(noise_rho) < 1e-12:
            X_noise = rng.normal(size=(n, n_noise))  # independent noise features
        else:
            h3 = rng.normal(size=n)
            X_noise = make_block(h3, n_noise, noise_rho)
        X = np.concatenate([X_strong, X_weak, X_noise], axis=1)
    else:
        X = np.concatenate([X_strong, X_weak], axis=1)

    # Standardize features (before building y won't change y because y only uses first 6)
    if standardize_features:
        X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-12)

    # Alias the first six for readability
    X1, X2, X3, X4, X5, X6 = [X[:, j] for j in range(6)]

    # Nonlinear target with interactions (noise features do not enter)
    strong_lin = 0.6*X1 + 0.4*X2 - 0.2*X3 + 0.1*X4
    strong_int = 1.2*(X1*X2) + 0.8*(X3*X4) - 0.6*(X1*X4)
    weak_part  = 0.15*(0.7*X5 + 0.3*X6) + 0.12*(X5*X6)

    g = np.tanh(strong_lin + strong_int) + 0.3*np.sin(0.5*X2 - 0.25*X3)
    y = g + weak_part + rng.normal(scale=noise_scale, size=n)

    if standardize_target:
        y = (y - y.mean()) / (y.std() + 1e-12)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    return X_train, X_test, y_train, y_test


X_train_12, X_test_12, y_train_12, y_test_12 = generate_correlated_data_noisy(n=10000, p=2, p_init=2, random_state=42, n_noise=0)

X_train_123, X_test_123, y_train_123, y_test_123 = generate_correlated_data_noisy(n=10000, p=3, p_init=2, random_state=42, n_noise=1)



In [3]:
from scipy.special import digamma
from sklearn.neighbors import NearestNeighbors

def mi_ksg_xy(X, y, k=5, metric='chebyshev', seed=0):
    X = np.asarray(X, float)
    y = np.asarray(y, float).reshape(-1, 1)
    n, d = X.shape

    Z = np.concatenate([X, y], axis=1)

    nbrs = NearestNeighbors(n_neighbors=k+1, metric=metric)
    nbrs.fit(Z)
    distances, _ = nbrs.kneighbors(Z, return_distance=True)
    eps = distances[:, k] - 1e-12
    eps = np.maximum(eps, 1e-15)

    nbrs_x = NearestNeighbors(metric=metric).fit(X)
    nx = np.array([len(idx)-1 for idx in nbrs_x.radius_neighbors(X, radius=eps, return_distance=False)], float)

    nbrs_y = NearestNeighbors(metric=metric).fit(y)
    ny = np.array([len(idx)-1 for idx in nbrs_y.radius_neighbors(y, radius=eps, return_distance=False)], float)

    mi = digamma(k) + digamma(n) - np.mean(digamma(nx+1) + digamma(ny+1))
    return float(mi)


In [ ]:
# X12 = X[:, :2]   # only the true signal
# X123 = X         # includes noise X3

mi_y_x12_k5  = mi_ksg_xy(X_train_12, y_train_12, k=5)
mi_y_x12_k10 = mi_ksg_xy(X_train_12, y_train_12, k=10)
mi_y_x123_k5 = mi_ksg_xy(X_train_123, y_train_123, k=5)
mi_y_x123_k10 = mi_ksg_xy(X_train_123, y_train_123, k=10)

print("KSG MI estimates (in nats):")
print(f"I(Y; X1,X2)      k=5  = {mi_y_x12_k5:.4f}")
print(f"I(Y; X1,X2)      k=10 = {mi_y_x12_k10:.4f}")
print(f"I(Y; X1,X2,X3)   k=5  = {mi_y_x123_k5:.4f}")
print(f"I(Y; X1,X2,X3)   k=10 = {mi_y_x123_k10:.4f}")
